## Lunar Lander con Deep Q Learning

## Integrantes

- Juan Jose Urioste
- Carlos Huerta Garcia
- Alejo Torres Teruel

## Contenido

En esta entregase presentan una implementacion de un agente que resuelve el problema de Lunar Lander utilizando Deep Q Learning y una implementacion de un agente que resuelve el mismo problema utilizando la tecninca de Double Deep Q Learning.

## Instalacion

Todas las dependencias necesarias estan listadas en el fichero `requirements.txt`. Que pueden ser instaladas con un manejador de paquetes de python como `pip` o `conda`.

## DevContainer

Ademas se provee una descripcion del Entorno de desarrollo utilizado para trabajar con el entorno de gymnasium.

como comando de post creacion se especifica la instalacion y compilacion de las dependencias necesarias para el entorno de gymnasium y el proyecto.

# Deep Q learning

In [5]:
from collections import deque
import os
import pathlib
import random
import shutil
from typing import Callable, List,  NamedTuple, Sequence, SupportsFloat, Union
import uuid

import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

## Configuracion de graficos
configuramos las dimensiones de los graficos para que se ajusten a las celdasdel notebook

In [6]:
%matplotlib notebook
plt.style.use('ggplot')
plt.rcParams.update({'figure.figsize': (15, 8),'figure.dpi': 64})

## Entorno de gymnasium
la variable env contendra una referencia al entorno donde se ejecutaran las acciones del agente, *env* contiene atributos y metodos que nos permiten interactuar con el entorno y obtener inforamcion sobre el espacio de obervaciones y acciones.

In [7]:
env = gym.make('LunarLander-v2')

## Informacion del entorno
Lo importante es comprender que existen 8 entradas que nos indican el estado de la nave en el entorno y cuatro posibles acciones a realizar.

En _Deep $Q$-learning_ los espacios de observaciones y acciones se corresponderán con las entradas y las salidas del modelo que sustituye a la tabla Q.

In [8]:
print(f'Input: {env.observation_space}')
print(f'Output: {env.action_space}')

Input: Box([-1.5       -1.5       -5.        -5.        -3.1415927 -5.
 -0.        -0.       ], [1.5       1.5       5.        5.        3.1415927 5.        1.
 1.       ], (8,), float32)
Output: Discrete(4)


In [9]:
#! Cerramos el entorno actual
env.close()

## Clase Transicion
representará la transición que ocurre de un estado $s_t$ a un estado $s_{t+1}$. Dicha transición contendrá la información de qué acción provocó la transición, cuál fue la recompensa de haberla llevado a cabo y si se ha terminado, ya sea por llegar a un estado terminal (p.ej. llegar al destino o morir) o por cualquier otra razón (p.ej llegar a un determinado límite de tiempo).

In [10]:
class Transition(NamedTuple):
    """Representa la transición de un estado al siguiente"""
    prev_state: gym.core.ObsType  # Estado origen de la transición
    next_state: gym.core.ObsType  # Estado destino de la transición
    action: gym.core.ActType      # Acción que provocó esta transición
    reward: SupportsFloat         # Recompensa obtenida
    terminated: bool              # Si se ha llegado a un estado terminal

## Clase Memoria
Nuestro agente, durante la ejecución del escenario, irá teniendo experiencias, representadas como transiciones entre estados. Esta memoria es la que se usará durante la fase de entrenamiento.

In [11]:
class Memory:
    """Representa la memoria de un agente.

    Concretamente, almacenará las últimas n transiciones realizadas en
    el entorno. El tamaño de la memoria se establecerá en el momento de
    crear la memoria del mismo.

    La memoria guarda las transiciones de manera ordenada, y se podrá
    acceder a ellos por índice, de manera que el recuerdo más lejano
    estará en la posición 0 y el más reciente en la posición -1.
    """

    def __init__(self, size: int):
        """Inicializa el objeto.

        :param size: El tamaño máximo de la memoria del agente."""
        self.max_size = int(size)
        self.transitions: deque = deque(maxlen=self.max_size)

    def append(self, transition: Transition):
        """Añade un nuevo recuerdo a la memoria del agente.

        :param transition: La transición a recordar."""
        self.transitions.append(transition)

    def batch(self, n: int) -> List[Transition]:
        """Devuelve n recuerdos aleatorios de la memoria.

        :param n: El número de recuerdos aleatorios a devolver. Si es
            superior al número de recuerdos totales devolverá todos los
            recuerdos almacenados.
        :returns: La lista de transiciones.
        """
        n = min(len(self.transitions), n)
        return random.sample(self.transitions, n)

    def __len__(self) -> int:
        """El número de recuerdos que contiene esta memoria.

        :returns: Un entero mayor o igual a 0.
        """
        return len(self.transitions)

    def __getitem__(
            self,
            key: Union[int, slice]
    ) -> Union[Transition, Sequence[Transition]]:
        """Devuelve el/los elemento/s especificados.

        :param key: El argumento que indica los elementos. Puede ser un
            entero normal o un slice.
        :returns: El/los elemento/s especificados por el índice.
        """
        return self.transitions.__getitem__(key)

## Agente de DQL

nuestro agente ademas contendra una estructura en de datos que servira como *memoria* en el se almacenaran las experiencias o transiciones previas del agente, esta memoria sera utilizada para entrenar la red neuronal.

## Concepto de Tarea

Llamaremos «tarea» a una instancia de un problema, que puede tratarse de algo episódica (tiene un comienzo y un fin) o continua (tiene comienzo pero no tiene fin).

El agente, para aprender, ejecutará tareas una detrás de otra. Cada una de ellas constará de un ciclo constante de percibir el entorno, decidir la acción a ejecutar y ejecutar dicha acción sobre el entorno. Esto lo gestionaremos a través del método `task` de nuestro agente. Es importante recalcar que cada tarea reiniciara el entorno.

## Percibir

Usaremos exclusivamente el estado devuelto por el entorno.

## Percibir

Este concepto se refiere a determinar en cada momento queé acción ejecutaremos dada una observación.

Usaremos la estrategia _$\epsilon$-greedy_ con decaimiento, de tal manera que no siempre escogerá la mejor acción, sino que variará dependiendo de en qué momento del entrenamiento estemos, comenzando con una estrategia más aleatoria al principio y más conservadora al final, ya que sabemos que no siempre la mejor accion es aquella que maximiza la recompensa a largo plazo, que es lo que buscamos.

## Accion

En este caso,delegamos totalmente la responsabilidad de actuar al entorno

## Modelo de comportamiento

La tabla $Q$ de la estrategia $Q$-learning se sustituye por un modelo de redes neuronales en _dee $Q$-learning_. Por tanto, habilitaremos al agente para que admita un modelo que será en el que almacenará su experiencia.

Para que sea más versátil, nuestro agente admitirá un modelo de tres formas diferentes:

1. Como función que devuelve un modelo.
1. Directamente como modelo
1. Como cadena de caracteres representando el _path_ donde se encuentra el modelo.

## Entrenamiento

La idea detrás del aprendizaje en este tipo de algoritmos es que el modelo de entrena cada $x$ pasos a partir de la información recabada del pasado, que está almacenada en memoria.

A continuación, se presena la clase del agente que implementada con Deep Q Learning

In [12]:
class Agent:
    def __init__(
            self, *,
            env: gym.Env,
            model: Union[Callable[[gym.Env], tf.keras.Model], tf.keras.Model, str],
            train_steps_rate,
            batch_size=32,
            memory_size: int = 1e5,
            gamma=0.99,
    ):
        self.env = env
        self.current_state = None
        self.memory = Memory(size=memory_size)
        self.train_steps_rate = train_steps_rate
        self.batch_size = batch_size
        self.gamma = gamma
        self.current_step = 0
        
        if callable(model):
            self.model = model(self.env)
        elif isinstance(model, tf.keras.models.Model):
            self.model = tf.keras.models.clone_model(model)
        elif isinstance(model, str):
            self.model = tf.keras.models.load_model(model)
        else:
            raise ValueError('Valid models are a function, a model or a path')

    def task(self, epsilon, max_iterations=None) -> float:
        self.current_state, _ = env.reset()
        self.current_step = 0
        max_iterations = max_iterations or np.inf
        reward = 0
        running = True
        while running and self.current_step < max_iterations:
            # Por si acaso tenemos un número máximo de acciones por tarea
            max_iterations -= 1
            
            perception = self.perceive()
            action = self.decide(perception,epsilon)
            transition = self.act(action)
            
            reward += transition.reward
            
            running = not transition.terminated
            self.current_step += 1
        return reward
      
    def perceive(self):
        return self.current_state
      
    def decide(self, perception: np.ndarray, epsilon: float) -> np.ndarray:
        if random.random() < epsilon: #! random
            return self.env.action_space.sample()
        else: #! seleccion voraz
            perception = perception[np.newaxis, ...]
            q_values = self.model.predict(perception, verbose=0)[0]
            return np.argmax(q_values)
    def act(self, action):
        next_state, reward, terminated, _, _ = self.env.step(action)
        self.current_state = next_state
        self.memory.append(Transition(
            prev_state=self.current_state,
            next_state=next_state,
            action=action,
            reward=reward,
            terminated=terminated,
        ))
        if len(self.memory) > self.batch_size and self.current_step % self.train_steps_rate == 0:
            batch = self.memory.batch(n=self.batch_size)
            inputs = np.array([t.prev_state for t in batch])
            labels = self.compute_labels(batch)
            self.model.fit(
                x=inputs,
                y=labels,
                batch_size=self.batch_size,
                verbose=0,
            )


        return self.memory[-1]

## Funcion Huber Loss 

La función de pérdida de Huber es bastante menos sensible a los valores atípicos que la MSE porque es exponencial sólo hasta cierto umbral, a partir del cual se convierte en lineal. Esto conduce que la convergencia del proceso de entrenamieto sea más estable.

In [13]:
def custom_huber_loss(delta):
    half = tf.constant(0.5, dtype=tf.keras.backend.floatx())

    def f(y, ŷ):
        # Máscara para todas aquellas acciones que no son 0 (es decir,
        # las que no provocaron la transición)
        mask = tf.cast(tf.not_equal(y, 0.0), tf.keras.backend.floatx())
        # Error cometido en aquellas acciones válidas (valor absoluto)
        error = tf.abs(y - ŷ) * mask
        # Dependiendo de delta, calculamos error cuadrático o lineal
        loss = tf.where(
            error < delta,
            half * tf.square(error),  # Error cuadrático
            delta * (error - half * delta)  # Error lineal
        )
        # El loss será la media de todos los valores distintos de cero
        return tf.reduce_sum(loss) / tf.reduce_sum(mask)

    f.__name__ = 'custom_huber_loss'
    return f

## Metodos auxiliares

Nos serviran para construir y cargar los modelos de redes neuronales

In [14]:
def build_model(env: gym.Env) -> tf.keras.models.Model:
    """Crea un nuevo modelo con nuestro agente."""
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(16, activation=tf.keras.layers.LeakyReLU(0.2), input_shape=env.observation_space.shape),
        tf.keras.layers.Dense(env.action_space.n, activation='linear'),
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss=custom_huber_loss(1))
    return model

def existing_model(task_num):
    def f():
        """Carga un modelo salvado previamente"""
        path = f'tmp/lunar-lander-{task_num}.h5'
        return tf.keras.models.load_model(path, custom_objects={
            'custom_huber_loss': custom_huber_loss(1)
        })
    return f

## Creacion del agente y parametros

In [15]:
  
state, _ = env.reset()
terminated = truncated = False
env = gym.make('LunarLander-v2')

agent = Agent(
    env=env,
    model=build_model,
    train_steps_rate=4,
    memory_size=2**16,
    batch_size=2**12,
    gamma=0.99,
)
fig,ax = plt.subplots(1,1)
ax.set_xlabel('Task')
ax.set_ylabel('Reward')
NUM_TASKS = 10
MAX_STEPS_PER_TASK = 1000

EPSILON_MAX = 1.
EPSILON_MIN = 0.01
EPSILON_DEC = EPSILON_MAX / NUM_TASKS

RUNNING_AVG_WINDOW_SIZE = 100

epsilon = EPSILON_MAX
rewards = []
rewards_avg = []



<IPython.core.display.Javascript object>

## Bucle de entrenamiento

In [16]:

for task in range(NUM_TASKS):
    # Reseteamos el entorno y el agente para comenzar un nuevo episodio
    reward = agent.task(epsilon=epsilon, max_iterations=MAX_STEPS_PER_TASK)

    agent.model.save(f"tmp/lunar-lander-{task}.h5")

    # Actualizamos el valor de epsilon
    epsilon = max(EPSILON_MIN, epsilon - EPSILON_DEC)

    rewards.append(reward)
    if len(rewards) < RUNNING_AVG_WINDOW_SIZE:
        rewards_avg.append(np.nan)
    else:
        rewards_avg.append(np.mean(rewards[-RUNNING_AVG_WINDOW_SIZE:]))
        
    #! Pintamos la gráfica
    ax.cla()
    ax.set_title(f'Task {task} (ε = {epsilon:5.4f}) reward = {reward:5.4f} (best {max(rewards):5.4f} on task {np.argmax(rewards)})')
    ax.plot(rewards, linewidth=0.5, label='Instant reward')
    ax.plot(rewards_avg, linewidth=1.5, label=f'{RUNNING_AVG_WINDOW_SIZE} steps average')
    ax.legend()
    fig.canvas.draw()
    tf.keras.backend.clear_session()

env.close()

## Prueba del agente

probamos el con mejor modelo obtenido durante el entrenamiento

In [17]:
BEST_TASK = np.argmax(rewards)

env = gym.make('LunarLander-v2', render_mode='human')
path = F'tmp/lunar-lander-{BEST_TASK}.h5'
model = tf.keras.models.load_model(path, custom_objects={
    'custom_huber_loss': custom_huber_loss(1)
})

state, _ = env.reset()

terminated = truncated = False
while not (terminated or truncated):
    action = np.argmax(model.predict(state[np.newaxis, ...], verbose=0)[0])
    state, _, terminated, truncated, _ = env.step(action)
env.close()


# Double Deep Q Learning
Acontinuacion se presenta una solucion al mismo problema pero utilizando Double Deep Q Learning
## Importamos las librerias necesarias

In [8]:
import random
import gymnasium as gym
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import numpy as np
from collections import deque


# Agente de DDQL

la clase DQLAgent se encargara de manejar la logica de entrenamiento y evaluacion del agente.
Va a depender del entorno sobre el cual se ejecute y agrupa los metodos y los atributos necesarios para implementar el algoritmo de DQL.Mas en especifico, en este caso se trata de DDQL ya que contamos con un modelo de target y otro de entrenamiento.


## Atributos

- `state_size`: dimension del espacio de estados
- `action_size`: dimension del espacio de acciones

Ambos son dependientes de entorno sobre el cual se ejecute el agente.

- `epsilon`: factor de exploracion
- `epsilon_decay`: factor de decaimiento de epsilon -> este factor se utiliza para actualziar el valor de epsilon en cada episodio esto con el fin de que el agente explore menos y explote mas a medida que avanza el entrenamiento
- `epsilon_min`: valor minimo de epsilon
- `learning_rate`: factor de aprendizaje -> a fin de cuenta nuestros modelos son modelos de Deep learning en este caso perceptrones multicapas con capas full conected por lo que el aprendizaje se realiza mediante el algoritmo de SGD y este factor es el que determina la magnitud de los cambios en los pesos de las conexiones de las neuronas
- `memory`: Es una estructura de datos optimizada para accesar los datos de sus extremos, en este caso se utiliza para almacenar las transiciones de la forma (estado, accion, recompensa, estado siguiente, done) y poder muestrear de forma aleatoria hacer que el modelo "reviva" experiencias pasadas y pueda aprender de ellas
- `model`: modelo de entrenamiento sobre el cual se va a realizar el aprendizaje
- `target_model`: modelo de target que se actualiza cada cierta cantidad de episodios y es aquel que se utiliza para predecir el valor de Q en el estado siguiente


## Metodos

- `build_model`: construye el modelo de Deep learning que se va a utilizar para entrenar el agente
- `remember`: almacena una transicion en la memoria del agente
- `act`: retorna la accion a tomar en el estado actual infiriendola en base al modelo
- `replay`: muestrea la memoria de manera aleatoria y crea un batch usando aquellas memorias que no terminaron el episodio, predecimos usando ambos modelos y sobre las memorias donde no termino el episodio actualizamos los valores Q de acuerdo a la formula de DQL, luego recuperamos las acciones desde el batch al igual que los targets que se actualizaran con el valor calculado anteriormente para luego entrenar el modelo. vamos a entrenar en un epoch ya que este procedimiento sucede en cada en cada step de cada episodio
- `adaptiveEGreedy`: Nos permite ajustar el factor de exploracion
- `targetModelUpdate`: Actualizamos los pesos del modelo de target con los pesos del modelo de entrenamiento. Ambos cuentan con la misma arquitectura


In [9]:
class DQLAgent:
    def __init__(self, env):
        self.state_size = env.observation_space.shape[0]
        self.action_size = env.action_space.n
        self.epsilon = 1
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9993
        self.gamma = 0.99
        self.learning_rate = 0.0001
        self.memory = deque(maxlen=4000)
        self.model = self.build_model()
        self.target_model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(64, input_dim=self.state_size, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, s):
        if np.random.rand() <= self.epsilon: #! epsilon greedy
            return np.random.choice(self.action_size)
        act_values = self.model.predict(s)
        return np.argmax(act_values[0])

    def replay(self,batch_size):
        if len(agent.memory) < batch_size: #! no tengo suficientes memorias
            return

        minibatch = random.sample(self.memory, batch_size) #! samplear la memoria tomando como tamaño batch_size
        minibatch = np.array(minibatch)
        not_done_indices = np.where(minibatch[:, 4] == False) #! me interesa saber los que no termino 
        y = np.copy(minibatch[:, 2])

        if len(not_done_indices[0]) > 0:
            predict_sprime = self.model.predict(np.vstack(minibatch[:, 3]))
            predict_sprime_target = self.target_model.predict(np.vstack(minibatch[:, 3]))
            
            y[not_done_indices] += np.multiply(self.gamma, predict_sprime_target[not_done_indices, np.argmax(predict_sprime[not_done_indices, :][0], axis=1)][0])

        actions = np.array(minibatch[:, 1], dtype=int)
        y_target = self.model.predict(np.vstack(minibatch[:, 0]))
        y_target[range(batch_size), actions] = y
        self.model.fit(np.vstack(minibatch[:, 0]), y_target, epochs=1, verbose=0)
        

            
    def adaptiveEGreedy(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def targetModelUpdate(self):
        self.target_model.set_weights(self.model.get_weights())
        

# Entrenamiento

- Creamos el entorno e instanciamos el agente
- definimos el tamaño del batch y la cantidad de episodios a entrenar
- Por cada episodio vamos a resetear el entorno y el estado del agente
- En cada step vamos a pedir una accion inferida por el a gente para ejecutarse en el entorno, luego memorizar la transicion y realizar el proceso de replay

- si el episodio finaliza vamos a actualizar el modelo de target

In [10]:
env = gym.make('LunarLander-v2')

state, _ = env.reset()

agent = DQLAgent(env)
state_number = env.observation_space.shape[0]

batch_size = 32
episodes = 20
for e in range(episodes):

    state, _ = env.reset()
    state = np.reshape(state, [1, state_number])
    total_reward = 0
    for time in range(1000):

        action = agent.act(state)

        next_state, reward, done, _, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_number])

        agent.remember(state, action, reward, next_state, done)

        state = next_state

        agent.replay(batch_size)

        total_reward += reward

        if done:
            agent.targetModelUpdate()
            break

    agent.adaptiveEGreedy()

    print('Episode: {}, Reward: {}'.format(e, total_reward))


1/1 [==============================] - 0s 66ms/step


/tmp/ipykernel_512/2808631621.py:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  minibatch = np.array(minibatch)


1/1 [==============================] - 0s 18ms/step
Episode: 0, Reward: -183.1831494361481
1/1 [==============================] - 0s 16ms/step
Episode: 1, Reward: -261.4352635944839
1/1 [==============================] - 0s 17ms/step
Episode: 2, Reward: -281.00924354746223
1/1 [==============================] - 0s 23ms/step
Episode: 3, Reward: -181.21122994313635
1/1 [==============================] - 0s 19ms/step
Episode: 4, Reward: -66.31232142171227
1/1 [==============================] - 0s 23ms/step
Episode: 5, Reward: -222.48652784172052
1/1 [==============================] - 0s 19ms/step
Episode: 6, Reward: -224.69782565774233
1/1 [==============================] - 0s 42ms/step
Episode: 7, Reward: -356.0019249853344
1/1 [==============================] - 0s 17ms/step
Episode: 8, Reward: -143.12822371661056
1/1 [==============================] - 0s 18ms/step
Episode: 9, Reward: -74.56627048153483
1/1 [==============================] - 0s 17ms/step
Episode: 10, Reward: -295.6105450

# Probamos el modelo
ejecutamos un bucle PPA pero con un entorno en modo de renderizado humano para poder visualizar el comportamiento del agente

In [ ]:
env = gym.make('LunarLander-v2', render_mode='human')

state, _ = env.reset()

terminated = truncated = False
while not (terminated or truncated):
    action = agent.act(state)
    state, _, terminated, truncated, _ = env.step(action)
env.close()